```{contents}
```
## Distributed Systems

### 1. Motivation & Intuition

Modern **Generative AI** models (LLMs, diffusion models, multimodal models) are **too large and too computationally intensive** to be trained or served on a single machine.

Distributed systems provide:

| Challenge     | Why Distribution is Required                       |
| ------------- | -------------------------------------------------- |
| Model size    | Models exceed single-GPU memory (100B+ parameters) |
| Training time | Months → days via parallelism                      |
| Dataset size  | Web-scale data cannot fit on one node              |
| Reliability   | Failures inevitable in large clusters              |
| Throughput    | Millions of inference requests                     |

**Key idea:**

> *Split computation, memory, and data across many machines and coordinate them efficiently.*

---

### 2. Core Components of a GenAI Distributed System

```
Users → API Gateway → Inference Cluster
                 ↘ Training Cluster
                 ↘ Data Storage Cluster
                 ↘ Parameter Servers / Checkpoint Store
```

| Layer         | Responsibility                               |
| ------------- | -------------------------------------------- |
| Compute nodes | GPUs/TPUs executing model code               |
| Networking    | High-speed interconnect (NVLink, InfiniBand) |
| Storage       | Object stores, distributed filesystems       |
| Coordination  | Schedulers, RPC, fault tolerance             |
| Orchestration | Kubernetes, Slurm, Ray                       |

---

### 3. Forms of Parallelism in Generative AI

| Type                     | What is split              | Used when                  |
| ------------------------ | -------------------------- | -------------------------- |
| **Data Parallelism**     | Training samples           | Large datasets             |
| **Model Parallelism**    | Model layers / tensors     | Models too big for one GPU |
| **Pipeline Parallelism** | Sequential stages of model | Deep transformer stacks    |
| **Tensor Parallelism**   | Matrix operations          | Transformer attention/MLP  |
| **Hybrid Parallelism**   | Combination                | Large-scale LLM training   |

#### Example: Hybrid Parallelism

```
Node Group 1: Data parallel
 ├─ GPU0: Layer1-4
 ├─ GPU1: Layer1-4
Node Group 2: Model parallel
 ├─ GPU2: Layer5-8
 ├─ GPU3: Layer5-8
```

---

### 4. Training Workflow

```
Dataset → Sharded Loader
       → Distributed Forward Pass
       → Gradient All-Reduce
       → Optimizer Update
       → Checkpoint Sync
```

**Communication primitives**

| Operation        | Purpose           |
| ---------------- | ----------------- |
| All-Reduce       | Average gradients |
| Broadcast        | Share parameters  |
| Scatter / Gather | Tensor sharding   |

---

### 5. Inference at Scale

| Technique             | Benefit                             |
| --------------------- | ----------------------------------- |
| Model sharding        | Serve models larger than GPU memory |
| KV-cache distribution | Efficient long-context inference    |
| Batching              | Maximize GPU utilization            |
| Speculative decoding  | Reduce latency                      |
| Autoscaling           | Handle variable traffic             |

```
Load Balancer → Inference Workers → KV Cache Store
```

---

### 6. Fault Tolerance & Reliability

| Mechanism            | Function                      |
| -------------------- | ----------------------------- |
| Checkpointing        | Resume training after failure |
| Replication          | High availability             |
| Heartbeat monitoring | Detect failures               |
| Elastic training     | Nodes can join/leave          |

---

### 7. Distributed Data Management

| Layer             | Tools                |
| ----------------- | -------------------- |
| Distributed FS    | HDFS, Ceph           |
| Object store      | S3, GCS              |
| Streaming         | Kafka, Pulsar        |
| Dataset pipelines | WebDataset, TFRecord |

---

### 8. Example: PyTorch Distributed Training

```python
import torch
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

dist.init_process_group("nccl")

model = MyTransformer().cuda()
model = DDP(model)

optimizer = torch.optim.AdamW(model.parameters())

for batch in loader:
    loss = model(batch).loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
```

---

### 9. Architecture Comparison

| Scale                | Typical Stack                |
| -------------------- | ---------------------------- |
| Single node          | PyTorch + CUDA               |
| Small cluster        | PyTorch DDP + NCCL           |
| Large cluster        | DeepSpeed, Megatron-LM, FSDP |
| Production inference | Ray, Triton, vLLM            |
| Cloud orchestration  | Kubernetes, Slurm            |

---

### 10. Why Distributed Systems Are Central to GenAI

Without distributed systems:

| Capability                    | Possible? |
| ----------------------------- | --------- |
| Train 100B+ LLM               | ❌         |
| Real-time global inference    | ❌         |
| Multi-modal foundation models | ❌         |
| Continuous learning           | ❌         |

Distributed systems are **the computational backbone** of modern Generative AI.

---

### 11. Mental Model Summary

> **Generative AI = Models × Data × Compute × Coordination**

Distributed systems provide the **coordination layer** that makes modern AI possible.

